# Lab 1 - Creating a Slice with a P4 Switch

This lab will walk you through creating an experiment that contains a a P4 programmable switch. The P4 switch that we will use is a software switch known as the behavioral model (BMv2).  Specifically, the experiment deploys a slice with three nodes: two end-hosts and a P4 switch. The topology is shown in the figure below

![Topology](./images/bmv2-p4-3switches.png)

# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

/Users/pranitgunjal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
fablib.delete_all()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "lab1"

In [ ]:
slice = fablib.new_slice(name="lab1")

### Step 3.2: Define the sites
The code below requests three sites from FABRIC: INDI, STAR, and NCSA

<img src="./labs_files/lab1/figs/fabric_sites.png" width="550px"><br>

In [ ]:
site1='SALT'
site2='STAR'
site3='ATLA'
site4='WASH'
site5='DALL'
site6='PSC'

print (f'The selected sites are {site1}, {site2}, {site3} {site4} {site5} {site6}') 

The selected sites are SALT, STAR, ATLA WASH DALL PSC


### Step 3.3: Creating the nodes
The code below creates three nodes: server1, server2 and server3. The servers (server1, server2 and server3) use the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1, server2 will be created on site2 and server3 will be created in site3


![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site2, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server3 = slice.add_node(name="server3", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')


### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

In [ ]:
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]
server3_iface = server3.add_component(model='NIC_Basic').get_interfaces()[0]

### Step 3.5: Creating a node for the P4 switches
The code below creates nodes that will run the P4 switches. The node use the following
<ul>
    <li> 16 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 40GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

- Switch1 will be created on site4
- Switch2 will be created on site5
- Switch3 will be created on site6


In [ ]:
# Add a node
switch1 = slice.add_node(name="switch1", 
                      site=site4, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

In [ ]:
# Add a node
switch2 = slice.add_node(name="switch2", 
                      site=site5, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

In [ ]:
# Add a node
switch3 = slice.add_node(name="switch3", 
                      site=site6, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

### Step 3.6: Adding two interfaces to the switch
The code below adds two Network Interface Cards (NICs) to the switch.

In [ ]:
switch1_iface1 = switch1.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch1_iface2 = switch1.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]
switch1_iface3 = switch1.add_component(model='NIC_Basic', name='net4_nic').get_interfaces()[0]

In [ ]:
switch2_iface1 = switch2.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]
switch2_iface2 = switch2.add_component(model='NIC_Basic', name='net3_nic').get_interfaces()[0]
switch2_iface3 = switch2.add_component(model='NIC_Basic', name='net5_nic').get_interfaces()[0]

In [ ]:
switch3_iface1 = switch3.add_component(model='NIC_Basic', name='net4_nic').get_interfaces()[0]
switch3_iface2 = switch3.add_component(model='NIC_Basic', name='net5_nic').get_interfaces()[0]
switch3_iface3 = switch3.add_component(model='NIC_Basic', name='net6_nic').get_interfaces()[0]

### Step 3.7: Connecting site1 and site2
Create a site-to-site network between 
- Server1 and Switch1
- Switch1 and Switch2
- Switch2 and Server2
- Switch1 and Switch3
- Switch2 and Switch3
- Switch3 and Server3

![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
net1 = slice.add_l2network(name='server1-switch1', interfaces=[server1_iface, switch1_iface1])

In [ ]:
net2 = slice.add_l2network(name='switch1-switch2', interfaces=[switch1_iface2, switch2_iface1])

In [ ]:
net3 = slice.add_l2network(name='server2-switch2', interfaces=[switch2_iface2, server2_iface])

In [ ]:
net4 = slice.add_l2network(name='switch1-switch3', interfaces=[switch1_iface3, switch3_iface1])

In [ ]:
net5 = slice.add_l2network(name='switch2-switch3', interfaces=[switch2_iface3, switch3_iface2])

In [ ]:
net6 = slice.add_l2network(name='server3-switch3', interfaces=[switch3_iface3, server3_iface])

### Step 3.9: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [ ]:
#Submit Slice Request
slice.submit();


Retry: 11, Time: 318 sec


ID,0618b848-a34b-4bf1-93e2-a38cd4932ce8
Name,lab1
Lease Expiration (UTC),2024-08-23 14:27:13 +0000
Lease Start (UTC),2024-08-22 14:27:13 +0000
Project ID,16eafd43-3e6c-4546-854c-935eea16552a
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
5387a269-a29f-4748-929b-8e6eaf85e778,server1,4,8,100,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:fea5:96b9,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fea5:96b9,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
83354303-a351-4d90-bfee-038fa70d5dfc,server2,4,8,100,default_ubuntu_20,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe47:f336,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe47:f336,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ea42f7a8-59e4-4125-806d-5f7b4dca6ad2,server3,4,8,100,default_ubuntu_20,qcow2,atla-w1.fabric-testbed.net,ATLA,ubuntu,2001:400:a100:3050:f816:3eff:fe77:b282,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3050:f816:3eff:fe77:b282,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
65c4d030-f58a-4bf3-b1ad-4b5df5305258,switch1,32,16,100,default_ubuntu_20,qcow2,wash-w3.fabric-testbed.net,WASH,ubuntu,2001:400:a100:3020:f816:3eff:fe73:9370,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3020:f816:3eff:fe73:9370,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
cff672ba-c13f-48ff-8694-49d9f67f7b0b,switch2,32,16,100,default_ubuntu_20,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe98:f68e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe98:f68e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
760c1dcc-5cfa-47e8-af57-6a80cbe6ae01,switch3,32,16,100,default_ubuntu_20,qcow2,psc-w3.fabric-testbed.net,PSC,ubuntu,2001:5e8:ff00:ffff:f816:3eff:fea0:67eb,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:5e8:ff00:ffff:f816:3eff:fea0:67eb,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
57ac6fb0-5b3b-4ca0-b442-deb2a905826b,server1-switch1,L2,L2STS,None,None,None,Active,
b330bfad-2941-44db-832f-07dfc540d634,server2-switch2,L2,L2STS,None,None,None,Active,
7019bd77-8b21-4e67-b9ff-42a7a3f93de2,server3-switch3,L2,L2STS,None,None,None,Active,
15a0c0eb-9ea8-4c60-b63e-91b49894d03b,switch1-switch2,L2,L2STS,None,None,None,Active,
67d0e46b-afe4-46e2-b6de-a69867f846ac,switch1-switch3,L2,L2STS,None,None,None,Active,
3e663239-0b35-49e3-8ee6-65fc0ba2cfe2,switch2-switch3,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
server1-None-p1,p1,server1,server1-switch1,100,config,,32:1A:72:75:55:6C,enp7s0,enp7s0,fe80::301a:72ff:fe75:556c,4,HundredGigE0/0/0/7
server2-None-p1,p1,server2,server2-switch2,100,config,,2A:77:C8:60:D3:BF,enp7s0,enp7s0,fe80::2877:c8ff:fe60:d3bf,4,HundredGigE0/0/0/9
server3-None-p1,p1,server3,server3-switch3,100,config,,0E:86:BD:78:2B:A1,enp7s0,enp7s0,fe80::c86:bdff:fe78:2ba1,4,HundredGigE0/0/0/5
switch1-net1_nic-p1,p1,switch1,server1-switch1,100,config,,0A:6B:BB:DB:8E:43,enp8s0,enp8s0,fe80::86b:bbff:fedb:8e43,4,HundredGigE0/0/0/9
switch1-net2_nic-p1,p1,switch1,switch1-switch2,100,config,,4E:6B:86:D3:40:40,enp9s0,enp9s0,fe80::4c6b:86ff:fed3:4040,4,HundredGigE0/0/0/9
switch1-net4_nic-p1,p1,switch1,switch1-switch3,100,config,,0A:53:98:C2:26:8E,enp7s0,enp7s0,fe80::853:98ff:fec2:268e,4,HundredGigE0/0/0/9
switch2-net2_nic-p1,p1,switch2,switch1-switch2,100,config,,16:BF:89:FA:01:6C,enp9s0,enp9s0,fe80::14bf:89ff:fefa:16c,4,HundredGigE0/0/0/7
switch2-net3_nic-p1,p1,switch2,server2-switch2,100,config,,16:38:40:86:F5:9C,enp7s0,enp7s0,fe80::1438:40ff:fe86:f59c,4,HundredGigE0/0/0/7
switch2-net5_nic-p1,p1,switch2,switch2-switch3,100,config,,16:78:EC:B7:76:89,enp8s0,enp8s0,fe80::1478:ecff:feb7:7689,4,HundredGigE0/0/0/7
switch3-net6_nic-p1,p1,switch3,server3-switch3,100,config,,0A:E9:B8:16:7A:89,enp9s0,enp9s0,fe80::8e9:b8ff:fe16:7a89,4,HundredGigE0/0/0/13



Time to print interfaces 362 seconds


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the BMv2 software switch and its control plane, the P4 compiler (p4c), and net-tools.

In [ ]:
slice = fablib.get_slice(name='lab1')

### Creating a Map to store all the important topology information

In [ ]:
topology = {}
topology['devices'] = {}

In [ ]:
server_names = ['server1', 'server2', 'server3']
switch_names = ['switch1', 'switch2', 'switch3']

for server_name in server_names:
    server = slice.get_node(name=server_name)
    topology['devices'][server_name] = {'node': server}
    topology['devices'][server_name]['type'] = 'server'

for switch_name in switch_names:
    switch = slice.get_node(name=switch_name)
    topology['devices'][switch_name] = {'node': switch}
    topology['devices'][switch_name]['type'] = 'switch'

In [ ]:
import pprint
from pprint import pprint as pp
pp(topology)

{'devices': {'server1': {'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156063b190>,
                         'type': 'server'},
             'server2': {'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15419e7b50>,
                         'type': 'server'},
             'server3': {'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15407d74d0>,
                         'type': 'server'},
             'switch1': {'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156065a690>,
                         'type': 'switch'},
             'switch2': {'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d1560638510>,
                         'type': 'switch'},
             'switch3': {'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d157006a290>,
                         'type': 'switch'}}}


In [ ]:
devices = topology['devices']

## Step 4.1 Installing BMv2
The BMv2 software switch will be installed on the switch nodes. We will upload the script [scripts/install_bmv2.sh](./scripts/install_bmv2.sh) to the switches and execute it

In [ ]:
switch1 = slice.get_node(name="switch1")     
switch1.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch1.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [ ]:
switch2 = slice.get_node(name="switch2")     
switch2.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch2.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [ ]:
switch3 = slice.get_node(name="switch3")     
switch3.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch3.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.2 Installing net-tools
The net-tools package will be installed on the switches, and server nodes. This package will allow us to use the *ifconfig* and the *arp* commands 

In [ ]:
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
server3 = slice.get_node(name="server3")
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server3.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch3.execute(f'sudo apt-get install -y net-tools', quiet=True)

# Step 5: Assigning IP and MAC addresses
In this step, we will assign IPv4 addresses to the interfaces of the servers and the switches. We will also hardcode the MAC addresses. 

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
node1_iface = server1.get_interface(network_name='server1-switch1') 
server1_iface_name = node1_iface.get_device_name()
print(f'server1_iface: {server1_iface_name}')

switch1_iface1 = switch1.get_interface(network_name='server1-switch1') 
switch1_iface1_name = switch1_iface1.get_device_name()
print(f'switch1_iface1: {switch1_iface1_name}')

switch1_iface2 = switch1.get_interface(network_name='switch1-switch2') 
switch1_iface2_name = switch1_iface2.get_device_name()
print(f'switch1_iface2: {switch1_iface2_name}')

switch1_iface3 = switch1.get_interface(network_name='switch1-switch3') 
switch1_iface3_name = switch1_iface3.get_device_name()
print(f'switch1_iface3: {switch1_iface3_name}')

switch2_iface1 = switch2.get_interface(network_name='switch1-switch2') 
switch2_iface1_name = switch2_iface1.get_device_name()
print(f'switch2_iface1: {switch2_iface1_name}')

switch2_iface2 = switch2.get_interface(network_name='server2-switch2') 
switch2_iface2_name = switch2_iface2.get_device_name()
print(f'switch2_iface2: {switch2_iface2_name}')

switch2_iface3 = switch2.get_interface(network_name='switch2-switch3') 
switch2_iface3_name = switch2_iface3.get_device_name()
print(f'switch2_iface3: {switch2_iface3_name}')

switch3_iface1 = switch3.get_interface(network_name='switch1-switch3') 
switch3_iface1_name = switch3_iface1.get_device_name()
print(f'switch3_iface1: {switch3_iface1_name}')

switch3_iface2 = switch3.get_interface(network_name='switch2-switch3') 
switch3_iface2_name = switch3_iface2.get_device_name()
print(f'switch3_iface2: {switch3_iface2_name}')

switch3_iface3 = switch3.get_interface(network_name='server3-switch3') 
switch3_iface3_name = switch3_iface3.get_device_name()
print(f'switch3_iface3: {switch3_iface3_name}')

node2_iface = server2.get_interface(network_name='server2-switch2') 
server2_iface_name = node2_iface.get_device_name()
print(f'server2_iface: {server2_iface_name}')

node3_iface = server3.get_interface(network_name='server3-switch3') 
server3_iface_name = node3_iface.get_device_name()
print(f'server3_iface: {server3_iface_name}')

server1_iface: enp7s0
switch1_iface1: enp8s0
switch1_iface2: enp9s0
switch1_iface3: enp7s0
switch2_iface1: enp9s0
switch2_iface2: enp7s0
switch2_iface3: enp8s0
switch3_iface1: enp7s0
switch3_iface2: enp8s0
switch3_iface3: enp9s0
server2_iface: enp7s0
server3_iface: enp7s0


## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

In [ ]:
for device in devices:
    node = devices[device]['node']
    ifaces = node.get_interfaces()
    topology['devices'][device]['interfaces'] = {}
    for iface in ifaces:
        network = iface.get_network()
        topology['devices'][device]['interfaces'][network.get_name()] = {'name': iface.get_device_name()}
pp(topology)

{'devices': {'server1': {'interfaces': {'server1-switch1': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156063b190>,
                         'type': 'server'},
             'server2': {'interfaces': {'server2-switch2': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15419e7b50>,
                         'type': 'server'},
             'server3': {'interfaces': {'server3-switch3': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15407d74d0>,
                         'type': 'server'},
             'switch1': {'interfaces': {'server1-switch1': {'name': 'enp8s0'},
                                        'switch1-switch2': {'name': 'enp9s0'},
                                        'switch1-switch3': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node objec

In [ ]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)
stdout, stderr = server3.execute(f'sudo ip link set dev {server3_iface_name} up', quiet=True)

stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface1_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface2_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface3_name} up', quiet=True)

stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface1_name} up', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface2_name} up', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface3_name} up', quiet=True)

stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface1_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface2_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface3_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>switch1_iface1_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
    <li>switch1_iface2_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
    <li>switch2_iface1_MAC = '00:00:00:00:00:04' (shown as 00:04 in the figure below)</li>
    <li>switch2_iface2_MAC = '00:00:00:00:00:05' (shown as 00:05 in the figure below)</li>
    <li>server2_iface_MAC = '00:00:00:00:00:06' (shown as 00:06 in the figure below)</li>
    <li>switch1_iface3_MAC = '00:00:00:00:00:07' (shown as 00:07 in the figure below)</li>
    <li>switch2_iface3_MAC = '00:00:00:00:00:08' (shown as 00:08 in the figure below)</li>
    <li>switch3_iface1_MAC = '00:00:00:00:00:09' (shown as 00:09 in the figure below)</li>
    <li>switch3_iface2_MAC = '00:00:00:00:00:10' (shown as 00:10 in the figure below)</li>
    <li>switch3_iface1_MAC = '00:00:00:00:00:11' (shown as 00:11 in the figure below)</li>
    <li> server3_iface_MAC = '00:00:00:00:00:12' (shown as 00:12 in the figure below) </li>

</ul>



![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
server1_iface_MAC = '00:00:00:00:00:01'

switch1_iface1_MAC = '00:00:00:00:00:02'
switch1_iface2_MAC = '00:00:00:00:00:03'

switch2_iface1_MAC = '00:00:00:00:00:04'
switch2_iface2_MAC = '00:00:00:00:00:05'

server2_iface_MAC = '00:00:00:00:00:06'

switch1_iface3_MAC = '00:00:00:00:00:07'
switch2_iface3_MAC = '00:00:00:00:00:08'

switch3_iface1_MAC = '00:00:00:00:00:09'
switch3_iface2_MAC = '00:00:00:00:00:10'

switch3_iface3_MAC = '00:00:00:00:00:11'

server3_iface_MAC = '00:00:00:00:00:12'


## Step 5.4 Configuring the IP and MAC addresses on server1_iface and switch_iface1

We will use
- Network 192.168.1.0/24 between Server1 (192.168.1.10) and Switch1 (192.168.1.1)
- Network 192.168.2.0/24 between Server2 (192.168.2.10) and Switch2 (192.168.2.1)
- Network 192.168.3.0/24 between Server3 (192.168.3.10) and Switch3 (192.168.3.1)


![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
server1_switch_subnet = "192.168.1.0/24"
server1_ip = '192.168.1.10/24'
switch1_ip1 = '192.168.1.1/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
topology['devices']['server1']['interfaces']['server1-switch1']['ip'] = server1_ip
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface1_name} {switch1_ip1}')
topology['devices']['switch1']['interfaces']['server1-switch1']['ip'] = switch1_ip1
stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
topology['devices']['server1']['interfaces']['server1-switch1']['MAC'] = server1_iface_MAC
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface1_name} hw ether {switch1_iface1_MAC}')
topology['devices']['switch1']['interfaces']['server1-switch1']['MAC'] = switch1_iface1_MAC
pp(topology)

{'devices': {'server1': {'interfaces': {'server1-switch1': {'MAC': '00:00:00:00:00:01',
                                                            'ip': '192.168.1.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156063b190>,
                         'type': 'server'},
             'server2': {'interfaces': {'server2-switch2': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15419e7b50>,
                         'type': 'server'},
             'server3': {'interfaces': {'server3-switch3': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15407d74d0>,
                         'type': 'server'},
             'switch1': {'interfaces': {'server1-switch1': {'MAC': '00:00:00:00:00:02',
                                                          

In [ ]:
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface2_name} hw ether {switch1_iface2_MAC}')
topology['devices']['switch1']['interfaces']['switch1-switch2']['MAC'] = switch1_iface2_MAC
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface1_name} hw ether {switch2_iface1_MAC}')
topology['devices']['switch2']['interfaces']['switch1-switch2']['MAC'] = switch2_iface1_MAC

In [ ]:
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface3_name} hw ether {switch1_iface3_MAC}')
topology['devices']['switch1']['interfaces']['switch1-switch3']['MAC'] = switch1_iface3_MAC
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface1_name} hw ether {switch3_iface1_MAC}')
topology['devices']['switch3']['interfaces']['switch1-switch3']['MAC'] = switch3_iface1_MAC
pp(topology)

{'devices': {'server1': {'interfaces': {'server1-switch1': {'MAC': '00:00:00:00:00:01',
                                                            'ip': '192.168.1.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156063b190>,
                         'type': 'server'},
             'server2': {'interfaces': {'server2-switch2': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15419e7b50>,
                         'type': 'server'},
             'server3': {'interfaces': {'server3-switch3': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15407d74d0>,
                         'type': 'server'},
             'switch1': {'interfaces': {'server1-switch1': {'MAC': '00:00:00:00:00:02',
                                                          

In [ ]:
server2_switch_subnet = "192.168.2.0/24"
server2_ip = '192.168.2.10/24'
switch2_ip2 = '192.168.2.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')
topology['devices']['server2']['interfaces']['server2-switch2']['ip'] = server2_ip
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface2_name} {switch2_ip2}')
topology['devices']['switch2']['interfaces']['server2-switch2']['ip'] = switch2_ip2
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')
topology['devices']['server2']['interfaces']['server2-switch2']['MAC'] = server2_iface_MAC
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface2_name} hw ether {switch2_iface2_MAC}')
topology['devices']['switch2']['interfaces']['server2-switch2']['MAC'] = switch2_iface2_MAC

In [ ]:
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface3_name} hw ether {switch2_iface3_MAC}')
topology['devices']['switch2']['interfaces']['switch2-switch3']['MAC'] = switch2_iface3_MAC
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface2_name} hw ether {switch3_iface2_MAC}')
topology['devices']['switch3']['interfaces']['switch2-switch3']['MAC'] = switch3_iface2_MAC
pp(topology)

{'devices': {'server1': {'interfaces': {'server1-switch1': {'MAC': '00:00:00:00:00:01',
                                                            'ip': '192.168.1.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156063b190>,
                         'type': 'server'},
             'server2': {'interfaces': {'server2-switch2': {'MAC': '00:00:00:00:00:06',
                                                            'ip': '192.168.2.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15419e7b50>,
                         'type': 'server'},
             'server3': {'interfaces': {'server3-switch3': {'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15407d74d0>,
                 

In [ ]:
server3_switch_subnet = "192.168.3.0/24"
server3_ip = '192.168.3.10/24'
switch3_ip2 = '192.168.3.1/24'

stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} {server3_ip}')
topology['devices']['server3']['interfaces']['server3-switch3']['ip'] = server3_ip
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface3_name} {switch3_ip2}')
topology['devices']['switch3']['interfaces']['server3-switch3']['ip'] = switch3_ip2

stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} hw ether {server3_iface_MAC}')
topology['devices']['server3']['interfaces']['server3-switch3']['MAC'] = server3_iface_MAC
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface3_name} hw ether {switch3_iface3_MAC}')
topology['devices']['switch3']['interfaces']['server3-switch3']['MAC'] = switch3_iface3_MAC
pp(topology)

{'devices': {'server1': {'interfaces': {'server1-switch1': {'MAC': '00:00:00:00:00:01',
                                                            'ip': '192.168.1.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156063b190>,
                         'type': 'server'},
             'server2': {'interfaces': {'server2-switch2': {'MAC': '00:00:00:00:00:06',
                                                            'ip': '192.168.2.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15419e7b50>,
                         'type': 'server'},
             'server3': {'interfaces': {'server3-switch3': {'MAC': '00:00:00:00:00:12',
                                                            'ip': '192.168.3.10/24',
                           

# Step 6: Configure forwarding and routing

## Step 6.1: Enable forwarding on the switch

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the switch device.

In [ ]:
stdout, stderr = switch1.execute('sudo sysctl -w net.ipv4.ip_forward=1' , quiet=True)

In [ ]:
stdout, stderr = switch2.execute('sudo sysctl -w net.ipv4.ip_forward=1' , quiet=True)

In [ ]:
stdout, stderr = switch3.execute('sudo sysctl -w net.ipv4.ip_forward=1' , quiet=True)

## Step 6.2: Delete routing entries for the routes to force traffic to go through the BMv2 switch

In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel.

In [ ]:
stdout, stderr = switch1.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip route del {server3_switch_subnet}', quiet=True)

In [ ]:
stdout, stderr = switch2.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip route del {server3_switch_subnet}', quiet=True)

In [ ]:
stdout, stderr = switch3.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip route del {server3_switch_subnet}', quiet=True)

## Step 6.3: Configure routing

In this step, we will configure static routes on server1 and server2. 
<ul>
    <li> For server1, we will add a route to reach the network 192.168.2.0/24 via 192.168.1.1 and 192.168.3.0/24 via 192.168.1.1</li>
    <li> For server2, we will add a route to reach the network 192.168.1.0/24 via 192.168.2.1 and 192.168.3.0/24 via 192.168.2.1</li>
    <li> For server3, we will add a route to reach the network 192.168.1.0/24 via 192.168.3.1 and 192.168.2.0/24 via 192.168.3.1</li>

</ul>


In [ ]:
gw1 = switch1_ip1.split('/')[0] # gw1 -> 192.168.1.1 
gw2 = switch2_ip2.split('/')[0] # gw2 -> 192.168.2.1
gw3 = switch3_ip2.split('/')[0] # gw3 -> 192.168.3.1

stdout, stderr = server1.execute(f'sudo ip route add {server2_switch_subnet} via {gw1}')
stdout, stderr = server1.execute(f'sudo ip route add {server3_switch_subnet} via {gw1}')


stdout, stderr = server2.execute(f'sudo ip route add {server1_switch_subnet} via {gw2}')
stdout, stderr = server2.execute(f'sudo ip route add {server3_switch_subnet} via {gw2}')

stdout, stderr = server3.execute(f'sudo ip route add {server1_switch_subnet} via {gw3}')
stdout, stderr = server3.execute(f'sudo ip route add {server2_switch_subnet} via {gw3}')


## Step 6.4: Configure ARP

In this step, we will configure static ARP entries on server1, server2 and server3. The reason we are doing this is because the switch does not process ARP packets unless programmed to. To make sure that ARP packets are not sent towards the switch, we will hardcode the MACs on the servers.

For each server, we will add an ARP entry to its switch's neighboring interface.

In [ ]:
print(f"sudo arp -s {gw1} {switch1_iface1_MAC}")
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {switch1_iface1_MAC}')

print(f"sudo arp -s {gw2} {switch2_iface2_MAC}")
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {switch2_iface2_MAC}')

print(f"sudo arp -s {gw3} {switch3_iface3_MAC}")
stdout, stderr = server3.execute(f'sudo arp -s {gw3} {switch3_iface3_MAC}')

sudo arp -s 192.168.1.1 00:00:00:00:00:02
sudo arp -s 192.168.2.1 00:00:00:00:00:05
sudo arp -s 192.168.3.1 00:00:00:00:00:11


# Step 7: Compiling and starting a simple P4 routing program

In this step, we will compile the P4 program and start the switch using the *p4c* compiler. To view and edit the P4 program, [Click Here](labs_files/lab1/main.p4)

The P4 program [main.p4](labs_files/lab1/main.p4) is located under lab_files/lab1/.

We will stop any previous instance of the switch (if any is existing) by killing its process.

Then, we will start the switch and allocate the interfaces to logical ports. 

![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
sw1_ifaces = devices['switch1']['interfaces']

for iface in sw1_ifaces:
    if sw1_ifaces[iface]['name'] == switch1_iface1_name:
        topology['devices']['switch1']['interfaces'][iface]['port'] = 0
    if sw1_ifaces[iface]['name'] == switch1_iface2_name:
        topology['devices']['switch1']['interfaces'][iface]['port'] = 1
    if sw1_ifaces[iface]['name'] == switch1_iface3_name:
        topology['devices']['switch1']['interfaces'][iface]['port'] = 2
        
sw2_ifaces = devices['switch2']['interfaces']

for iface in sw2_ifaces:
    if sw2_ifaces[iface]['name'] == switch2_iface1_name:
        topology['devices']['switch2']['interfaces'][iface]['port'] = 0
    if sw2_ifaces[iface]['name'] == switch2_iface2_name:
        topology['devices']['switch2']['interfaces'][iface]['port'] = 1
    if sw2_ifaces[iface]['name'] == switch2_iface3_name:
        topology['devices']['switch2']['interfaces'][iface]['port'] = 2

sw3_ifaces = devices['switch3']['interfaces']

for iface in sw3_ifaces:
    if sw3_ifaces[iface]['name'] == switch3_iface1_name:
        topology['devices']['switch3']['interfaces'][iface]['port'] = 0
    if sw3_ifaces[iface]['name'] == switch3_iface2_name:
        topology['devices']['switch3']['interfaces'][iface]['port'] = 1
    if sw3_ifaces[iface]['name'] == switch3_iface3_name:
        topology['devices']['switch3']['interfaces'][iface]['port'] = 2

pp(topology)

{'devices': {'server1': {'interfaces': {'server1-switch1': {'MAC': '00:00:00:00:00:01',
                                                            'ip': '192.168.1.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156063b190>,
                         'type': 'server'},
             'server2': {'interfaces': {'server2-switch2': {'MAC': '00:00:00:00:00:06',
                                                            'ip': '192.168.2.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15419e7b50>,
                         'type': 'server'},
             'server3': {'interfaces': {'server3-switch3': {'MAC': '00:00:00:00:00:12',
                                                            'ip': '192.168.3.10/24',
                           

In [ ]:
for i in range(1, 4):
    topology['devices'][f'switch{i}']['subnet'] = f'192.168.{i}.0/24'
pp(topology)

{'devices': {'server1': {'interfaces': {'server1-switch1': {'MAC': '00:00:00:00:00:01',
                                                            'ip': '192.168.1.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d156063b190>,
                         'type': 'server'},
             'server2': {'interfaces': {'server2-switch2': {'MAC': '00:00:00:00:00:06',
                                                            'ip': '192.168.2.10/24',
                                                            'name': 'enp7s0'}},
                         'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d15419e7b50>,
                         'type': 'server'},
             'server3': {'interfaces': {'server3-switch3': {'MAC': '00:00:00:00:00:12',
                                                            'ip': '192.168.3.10/24',
                           

In [ ]:
switch1.upload_file('labs_files/main.p4', 'main.p4')
stdout, stderr = switch1.execute(f'p4c main.p4')
stdout, stderr = switch1.execute(f'sudo pkill simple_switch')
stdout, stderr = switch1.execute(f'sudo simple_switch -i 0@{switch1_iface1_name} -i 1@{switch1_iface2_name} -i 2@{switch1_iface3_name} main.json &')

Calling target program-options parser
Adding interface enp8s0 as port 0
Adding interface enp9s0 as port 1
Adding interface enp7s0 as port 2


In [ ]:
switch2.upload_file('labs_files/main.p4', 'main.p4')
stdout, stderr = switch2.execute(f'p4c main.p4')
stdout, stderr = switch2.execute(f'sudo pkill simple_switch')
stdout, stderr = switch2.execute(f'sudo simple_switch -i 0@{switch2_iface1_name} -i 1@{switch2_iface2_name} -i 2@{switch2_iface3_name} main.json &')

Calling target program-options parser
Adding interface enp9s0 as port 0
Adding interface enp7s0 as port 1
Adding interface enp8s0 as port 2


In [ ]:
switch3.upload_file('labs_files/main.p4', 'main.p4')
stdout, stderr = switch3.execute(f'p4c main.p4')
stdout, stderr = switch3.execute(f'sudo pkill simple_switch')
stdout, stderr = switch3.execute(f'sudo simple_switch -i 0@{switch3_iface1_name} -i 1@{switch3_iface2_name} -i 2@{switch3_iface3_name} main.json &')

Calling target program-options parser
Adding interface enp7s0 as port 0
Adding interface enp8s0 as port 1
Adding interface enp9s0 as port 2


# Disable Offloading for Higher Rates

In [ ]:
for name in switch_names:
    node = devices[name]['node']
    ifaces = devices[name]['interfaces']
    node.upload_file('scripts/disable_offload.sh', 'disable_offload.sh')
    for iface in ifaces:
        iface_name = devices[name]['interfaces'][iface]['name']
        node.execute(f'sudo chmod +x ./disable_offload.sh && sudo ./disable_offload.sh {iface_name}', quiet=True)
        pp(name + " " + iface_name)

'switch1 enp8s0'
'switch1 enp9s0'
'switch1 enp7s0'
'switch2 enp9s0'
'switch2 enp7s0'
'switch2 enp8s0'
'switch3 enp9s0'
'switch3 enp7s0'
'switch3 enp8s0'


## Step 7.1: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

In [ ]:
slice = fablib.get_slice("lab1")
switch1 = slice.get_node("switch1")
switch2 = slice.get_node("switch2")
switch3 = slice.get_node("switch3")

In [ ]:
switch1.upload_file('labs_files/rules1.sh', 'rules1.sh')
stdout, stderr = switch1.execute('chmod +x rules1.sh && ./rules1.sh')

Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:01:0a/32
action:              MyIngress.forward
runtime data:        00:00:00:00:00:01	00:00
Entry has been added with handle 0
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:02:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:04	00:01
Entry has been added with handle 1
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:03:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:09	00:02
Entry has been added with handle 2
RuntimeCmd: 


In [ ]:
switch2.upload_file('labs_files/rules2.sh', 'rules2.sh')
stdout, stderr = switch2.execute('chmod +x rules2.sh && ./rules2.sh')

Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:01:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:03	00:00
Entry has been added with handle 0
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:02:0a/32
action:              MyIngress.forward
runtime data:        00:00:00:00:00:06	00:01
Entry has been added with handle 1
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:03:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:10	00:02
Entry has been added with handle 2
RuntimeCmd: 


In [ ]:
switch3.upload_file('labs_files/rules3.sh', 'rules3.sh')
stdout, stderr = switch3.execute('chmod +x rules3.sh && ./rules3.sh')

Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:01:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:07	00:00
Entry has been added with handle 0
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:02:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:08	00:01
Entry has been added with handle 1
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:03:0a/32
action:              MyIngress.forward
runtime data:        00:00:00:00:00:12	00:02
Entry has been added with handle 2
RuntimeCmd: 


In [ ]:
rules = {}
rules['switch1'] = {}
rules['switch2'] = {}
rules['switch3'] = {}

### Creating a Dictionary to store current rules

- Dictionary to store the current rules in all the tables
- Can potentially add this to the topology dictionary to simplify things

In [ ]:
rules1 = open('labs_files/rules1.sh', 'r')
for i, line in enumerate(rules1):
    line = line[:-1]
    words = line.split(" ")
    print(words[4] + " " + words[6] + " " + words[7][:-1])
    rules['switch1'][i] = {
        'dest_ip': words[4],
        'dest_MAC': words[6],
        'egress_port': words[7][:-1]
    }
rules2 = open('labs_files/rules2.sh', 'r')
for i, line in enumerate(rules2):
    line = line[:-1]
    words = line.split(" ")
    print(words[4] + " " + words[6] + " " + words[7][:-1])
    rules['switch2'][i] = {
        'dest_ip': words[4],
        'dest_MAC': words[6],
        'egress_port': words[7][:-1]
    }

rules3 = open('labs_files/rules3.sh', 'r')
for i, line in enumerate(rules3):
    line = line[:-1]
    words = line.split(" ")
    print(words[4] + " " + words[6] + " " + words[7][:-1])
    rules['switch3'][i] = {
        'dest_ip': words[4],
        'dest_MAC': words[6],
        'egress_port': words[7][:-1]
    }
pp(rules)

192.168.1.10/32 00:00:00:00:00:01 0
192.168.2.0/24 00:00:00:00:00:04 1
192.168.3.0/24 00:00:00:00:00:09 2
192.168.1.0/24 00:00:00:00:00:03 0
192.168.2.10/32 00:00:00:00:00:06 1
192.168.3.0/24 00:00:00:00:00:10 2
192.168.1.0/24 00:00:00:00:00:07 0
192.168.2.0/24 00:00:00:00:00:08 1
192.168.3.10/32 00:00:00:00:00:12 2
{'rules1': {0: {'dest_MAC': '00:00:00:00:00:01',
                'dest_ip': '192.168.1.10/32',
                'egress_port': '0'},
            1: {'dest_MAC': '00:00:00:00:00:04',
                'dest_ip': '192.168.2.0/24',
                'egress_port': '1'},
            2: {'dest_MAC': '00:00:00:00:00:09',
                'dest_ip': '192.168.3.0/24',
                'egress_port': '2'}},
 'rules2': {0: {'dest_MAC': '00:00:00:00:00:03',
                'dest_ip': '192.168.1.0/24',
                'egress_port': '0'},
            1: {'dest_MAC': '00:00:00:00:00:06',
                'dest_ip': '192.168.2.10/32',
                'egress_port': '1'},
            2: {'dest_MA

## Step 7.2: Running a ping test

In [ ]:
server2_dest_ip = server2_ip.split('/')[0]
stdout, stderr = server1.execute(f'ping {server2_dest_ip} -c 3')

PING 192.168.2.10 (192.168.2.10) 56(84) bytes of data.
64 bytes from 192.168.2.10: icmp_seq=1 ttl=62 time=105 ms
64 bytes from 192.168.2.10: icmp_seq=2 ttl=62 time=103 ms
64 bytes from 192.168.2.10: icmp_seq=3 ttl=62 time=103 ms

--- 192.168.2.10 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2002ms
rtt min/avg/max/mdev = 103.110/103.825/104.951/0.805 ms


In [ ]:
server3_dest_ip = server3_ip.split('/')[0]
stdout, stderr = server1.execute(f'ping {server3_dest_ip} -c 3')

PING 192.168.3.10 (192.168.3.10) 56(84) bytes of data.
64 bytes from 192.168.3.10: icmp_seq=1 ttl=62 time=71.7 ms
64 bytes from 192.168.3.10: icmp_seq=2 ttl=62 time=70.9 ms
64 bytes from 192.168.3.10: icmp_seq=3 ttl=62 time=71.1 ms

--- 192.168.3.10 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2004ms
rtt min/avg/max/mdev = 70.896/71.215/71.688/0.340 ms


In [ ]:
server3_dest_ip = server3_ip.split('/')[0]
stdout, stderr = server2.execute(f'ping {server3_dest_ip} -c 3')

PING 192.168.3.10 (192.168.3.10) 56(84) bytes of data.
64 bytes from 192.168.3.10: icmp_seq=1 ttl=62 time=84.0 ms
64 bytes from 192.168.3.10: icmp_seq=2 ttl=62 time=83.3 ms
64 bytes from 192.168.3.10: icmp_seq=3 ttl=62 time=83.5 ms

--- 192.168.3.10 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2002ms
rtt min/avg/max/mdev = 83.331/83.623/84.043/0.304 ms


# New Rules

In [ ]:
pp(topology)

In [ ]:
stdout, stderr = server1.execute(f'sudo apt-get update && sudo apt-get install -y iperf3', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get update && sudo apt-get install -y iperf3', quiet=True)
stdout, stderr = server3.execute(f'sudo apt-get update && sudo apt-get install -y iperf3', quiet=True)

In [ ]:
import json
import numpy
temp = None

### Creating a Graph to be able to implement Djikstra's algorithm on the topology

- The edges are updated using iperf. Will need to find a way to monitor traffic flow between switches
    - Thinking of using scapy

In [ ]:
graph = {}
subnets = ['192.168.1.10', '192.168.2.10', '192.168.3.10']
for i, src in enumerate(server_names):
    src_node = devices[src]['node']
    src_ip = devices[src]['interfaces'][f'server{i+1}-switch{i+1}']['ip']
    src_name = switch_names[i]
    graph[src_name] = {}
    for j, dst in enumerate(server_names):
        if i != j:
            dst_ip = devices[dst]['interfaces'][f'server{j+1}-switch{j+1}']['ip']
            dst_ip = dst_ip.split('/')[0]
            dst_name = switch_names[j]
            dst_node = devices[dst]['node']

            dst_node.execute_thread('iperf3 -s -1')
            stdout, stderr = src_node.execute(f'iperf3 -c {dst_ip} -J', quiet=True)
            
            temp = tojson = json.loads(stdout)
            while 'error' in tojson:
                stdout, stderr = src_node.execute(f'iperf3 -c {dst_ip} -J', quiet=True)
                temp = tojson = json.loads(stdout)
            print(src + " " + dst + " " + str(tojson['end']['sum_received']['bits_per_second']))
            cleaned_bit = 1 / tojson['end']['sum_received']['bits_per_second']
            
            '''data = []
            for x, interval in enumerate(tojson['intervals']):
                data.append(tojson['intervals'][x]['streams'][0]['bits_per_second'])
            print(numpy.mean(data))
            print(data) '''
            graph[src_name][dst_name] = cleaned_bit

pp(graph) 

server1 server2 5728.024269733771
server1 server3 298555.01411223196
server2 server1 4579.427207901409
server2 server3 286539.10909354035
server3 server1 309910.3294163838
server3 server2 290508.3553248562
{'switch1': {'switch2': 0.0001745802658839779,
             'switch3': 3.3494664391202717e-06},
 'switch2': {'switch1': 0.00021836792127071826,
             'switch3': 3.489924998941597e-06},
 'switch3': {'switch1': 3.2267398182021798e-06,
             'switch2': 3.442241786408403e-06}}


In [ ]:
graph['server1'] = {'switch1': 0}
graph['server2'] = {'switch2': 0}
graph['server3'] = {'switch3': 0}
graph['switch1']['server1'] = 0
graph['switch2']['server2'] = 0
graph['switch3']['server3'] = 0
pp(graph)

{'server1': {'switch1': 0},
 'server2': {'switch2': 0},
 'server3': {'switch3': 0},
 'switch1': {'server1': 0,
             'switch2': 0.0001745802658839779,
             'switch3': 3.3494664391202717e-06},
 'switch2': {'server2': 0,
             'switch1': 0.00021836792127071826,
             'switch3': 3.489924998941597e-06},
 'switch3': {'server3': 0,
             'switch1': 3.2267398182021798e-06,
             'switch2': 3.442241786408403e-06}}


In [ ]:
import sys
from heapq import heapify, heappush, heappop

### Adding a delay between switch1 and switch2

In [ ]:
switch1.execute(f'sudo tc qdisc add dev {devices['switch1']['interfaces']['switch1-switch2']['name']} root netem delay 10000ms', quiet=True)

('', '')

### Djikstra Implementation

- Utilizes a priority queue to get the next shortest paths to explore
- Runs in $O(E \cdot log(N) + N)$

In [ ]:
def dijkstra(graph, src, dest):
    node_data = {}
    for device in devices:
        node_data[device] = {
            'cost': float('inf'),
            'prev': None
        }
    node_data[src]['dist'] = 0
    pq = []
    heappush(pq, (0, src))

    visited = set()
    while pq:
        curr_dist, curr_node = heappop(pq)
        
        if curr_node in visited:
            continue
        visited.add(curr_node)
        if curr_node == dest:
            break
        
        for neighbor, weight in graph[curr_node].items():
            new_dist = curr_dist + weight

            if new_dist < node_data[neighbor]['cost']:
                node_data[neighbor]['cost'] = new_dist
                node_data[neighbor]['prev'] = curr_node
                heappush(pq, (new_dist, neighbor))
    
    path = []
    curr_node = dest
    while curr_node is not None:
        path.append(curr_node)
        if curr_node == src:
            break
        curr_node = node_data[curr_node]['prev']
     

    path.reverse()
    return path
        


In [ ]:
res = dijkstra(graph, 'server1', 'server2')
print(res)

['server1', 'switch1', 'switch3', 'switch2', 'server2']


### Updating routes based on the results from the Djikstra's Algorithm

- Takes the route information and updates the routes in the switches as need be
- Do this by checking current rules and update them as need be

In [ ]:
route = res[1:len(res)-1]
dst = route[-1]
src = route[0]
i = 0
for i in range(len(route)-1):
    sw1, sw2 = route[i], route[i+1]
    
    node = devices[sw1]['node']
    num = sw1[-1]
    for h in rules[f'switch{num}']:
        handle = MAC = egress = None
        if rules[f'switch{num}'][h]['dest_ip'] == devices[dst]['subnet']:
            handle = h
            net1 = f'{sw1}-{sw2}'
            net2 = f'{sw2}-{sw1}'
            if net1 in devices[sw2]['interfaces']:
                MAC = devices[sw2]['interfaces'][net1]['MAC']
                egress = devices[sw1]['interfaces'][net1]['port']
            elif net2 in devices[sw2]['interfaces']:
                MAC = devices[sw2]['interfaces'][net2]['MAC']
                egress = devices[sw1]['interfaces'][net2]['port']
            
            print(f'{handle} {MAC} {egress}')
            command = f'echo \"table_modify MyIngress.ipv4_lpm MyIngress.forward {handle} {MAC} {egress}\" | simple_switch_CLI'
            node.execute(command)
            rules[f'switch{num}'][handle]['dest_MAC'] = MAC
            rules[f'switch{num}'][handle]['egress_port'] = egress
            break

1 00:00:00:00:00:09 2
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Modifying entry 1 for lpm match table MyIngress.ipv4_lpm
RuntimeCmd: 
1 00:00:00:00:00:08 1
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Modifying entry 1 for lpm match table MyIngress.ipv4_lpm
RuntimeCmd: 


# Generating New Graphs

In [ ]:
server3.execute_thread('iperf3 -s -1')
stdout, stderr = server1.execute('iperf3 -c 192.168.3.10 -J -t 50', quiet=True)
orig = json.loads(stdout)

In [ ]:
orig_data = []
for j in range(len(orig['intervals'])):
    orig_data.append(orig['intervals'][j]['streams'][0]['bits_per_second'])


In [ ]:
import matplotlib.pyplot as plt

x = [i for i in range(1, 51)]
orig_y = []
for test in orig_data:
    orig_y.append(test / (10**6))
plt.plot(x, orig_y)

In [ ]:
server3.execute_thread('iperf3 -s -1')
stdout, stderr = server1.execute('iperf3 -c 192.168.3.10 -J -t 50', quiet=True)
new = json.loads(stdout)

In [ ]:
new_data = []
for j in range(len(new['intervals'])):
    new_data.append(new['intervals'][j]['streams'][0]['bits_per_second'])
print(new_data)

In [ ]:
import matplotlib.pyplot as plt

x = [i for i in range(1, 51)]
new_y = []
for test in new_data:
    new_y.append(test / (10**6))

In [ ]:
plt.plot(x, new_y)
plt.plot(x, orig_y)
plt.legend(['Dynamic Routing', 'No intervention'])
plt.xlabel('Time (seconds)')
plt.ylabel('Bitrate (Mbps)')
plt.title('Network Efficiency with Intervention vs No Intervention')

plt.savefig('model_intervension.jpg')

## Step 9: Delete the Slice

This concludes Lab 1. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="lab1")
slice.delete()